# add Area, SD, HD columns to supersite list
- read in pct_supersite.xlsx
- group by supersite
- extract list of Area, SD, HD

## Possible enhancements
- Transform Areas column from Python list to string, eg. [BO-03, BO-04, MT-03] to "BO-03, BO-04, MT-03"
- Transform Areas column from AreasShort to AreasLong format e.g. "Boulder 03"

- create list of Supersites for each SD, HD

In [1]:
import pandas as pd

In [5]:
# read in pct_supersite.xlsx

cols = ['Pct', 'AreaShort', 'SD', 'HD', 'supersite']
pctss_1_21 = pd.read_excel('data/pct_supersite.xlsx', usecols=cols)

pctss_1_21.info()
pctss_1_21

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pct        193 non-null    int64 
 1   AreaShort  193 non-null    object
 2   SD         193 non-null    int64 
 3   HD         193 non-null    int64 
 4   supersite  193 non-null    object
dtypes: int64(3), object(2)
memory usage: 7.7+ KB


,Pct,AreaShort,SD,HD,supersite
0,2,LM-04,15,19,Trail Ridge MS
1,3,SU-01,18,49,Eldorado K8
2,4,BO-01,15,12,Manhattan MS
3,100,SU-01,18,12,Eldorado K8
4,101,SU-01,18,12,Eldorado K8
...,...,...,...,...,...
188,912,MT-02,15,49,Centennial MS
189,913,MT-02,15,49,Jamestown School
190,914,MT-03,15,49,Allenspark Fire Station
191,915,MT-03,15,49,Lyons Middle Senior


In [2]:
# read in pct_supersite.xlsx

cols = ['Pct', 'AreaShort', 'SD', 'HD', 'supersite']
pctss = pd.read_excel('data/pct_supersite.xlsx', usecols=cols)

pctss.info()
pctss

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pct        193 non-null    int64 
 1   AreaShort  193 non-null    object
 2   SD         193 non-null    int64 
 3   HD         193 non-null    int64 
 4   supersite  193 non-null    object
dtypes: int64(3), object(2)
memory usage: 7.7+ KB


,Pct,AreaShort,SD,HD,supersite
0,2,LM-04,15,19,Trail Ridge MS
1,3,SU-01,18,49,Eldorado K8
2,4,BO-01,15,12,Manhattan MS
3,100,SU-01,18,12,Eldorado K8
4,101,SU-01,18,12,Eldorado K8
...,...,...,...,...,...
188,912,MT-02,15,49,Centennial MS
189,913,MT-02,15,49,Jamestown School
190,914,MT-03,15,49,Allenspark Fire Station
191,915,MT-03,15,49,Lyons Middle Senior


In [3]:
pctss.tail(20)

,Pct,AreaShort,SD,HD,supersite
173,855,BO-01,18,49,Southern Hills MS
174,856,BO-01,18,49,Southern Hills MS
175,857,BO-01,18,49,Southern Hills MS
176,900,BO-01,15,49,Southern Hills MS
177,901,MT-01,15,49,Nederland HS
178,902,MT-01,15,49,Nederland HS
179,903,MT-01,15,49,Nederland HS
180,904,MT-01,15,49,Nederland HS
181,905,MT-01,15,49,Nederland HS
182,906,BO-01,15,49,Southern Hills MS


In [ ]:
## try create arealist using Pandas functions for Altona MS

# type(pctss[pctss['supersite'] == 'Altona MS']['AreaShort']) # .unique().tolist()

pctss[pctss['supersite'] == 'Altona MS']['AreaShort'].unique().tolist()
# pctss[pctss['supersite'] == 'Altona MS']['SD'].unique().tolist()


In [ ]:
def collist(s):
    return s.unique().tolist()

ss_areas = pctss.groupby('supersite').agg(
    areas=('AreaShort', collist),
    SDs=('SD', collist),
    HDs=('HD', collist),
    PCTs=('Pct', collist)
)

ss_areas.to_excel('data/supersite_areas.xlsx', sheet_name='Areas, SD, HD at Supersite', index=True)

ss_areas.info()
ss_areas



## Create list of Supersites by SD, HD
- SD18 at Niwot, New Vista, Monarch, Eldorado K8, Centaurus, ...

In [ ]:
pctss.info()


hd_ss = pctss.groupby('HD').agg(
    supersites=('supersite', collist),
    PCTs=('Pct', collist)
)

hd_ss.to_excel('data/hd_ss.xlsx', sheet_name='HDs', index=True)

hd_ss

In [ ]:

sd_ss = pctss.groupby('SD').agg(
    supersites=('supersite', collist),
    PCTs=('Pct', collist)
)

sd_ss.to_excel('data/sd_ss.xlsx', sheet_name='SDs', index=True)

sd_ss

In [15]:
with pd.ExcelWriter('data/supersite_area_HD_SD_DRAFT_20240121.xlsx') as writer:
    
    ss_areas.to_excel(writer, sheet_name='Areas, SD, HD at Supersite', index=True)
    hd_ss.to_excel(writer, sheet_name="HDs")  
    sd_ss.to_excel(writer, sheet_name="SDs")  